In [1]:
#필요한 모듈 import

import numpy as np
from sklearn.mixture import GaussianMixture
import glob
import os
import wave
from scipy.io import wavfile
import pickle
import matplotlib.pyplot as plt
import librosa
import librosa.effects as effects
import librosa.util
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import joblib

In [2]:
# RAW 파일을 WAV 파일로 변환하는 함수
def convert_raw_to_wav(raw_filepath, wav_filepath):
    sample_width = 2  # 16비트 샘플
    sample_rate = 16000  # 샘플링 속도
    num_channels = 1  # 모노 오디오

    # RAW 파일 읽기
    with open(raw_filepath, 'rb') as raw_file:
        raw_data = raw_file.read()

    # WAV 파일 생성
    with wave.open(wav_filepath, 'wb') as wav_file:
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(sample_rate)
        wav_file.setnchannels(num_channels)

        # RAW 데이터를 WAV 파일에 쓰기
        wav_file.writeframes(raw_data)
        
def extract_features(audio_path):
    # 음성 신호 불러오기
    sr=16000
    audio, sr = librosa.load(audio_path, sr=sr)
    n_fft = 8192  # 원하는 FFT 윈도우 크기
    hop_length=4096
    n_mfcc=75
    fmin=0
    fmax=8000
    top_db=90

    # 스케일 조정
    max_amp = np.max(np.abs(audio))
    scale_factor = 1.0 / max_amp
    scaled_audio = audio * scale_factor
     
    # 스펙트로그램 평활화
    norm_audio = librosa.util.normalize(scaled_audio)
    
    # RMS 정규화
    rms = librosa.feature.rms(y=norm_audio)
    normalized_audio = norm_audio / (np.max(rms) + 1e-8)
  
    # MFCC 추출
    mfcc = librosa.feature.mfcc(y=normalized_audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft, fmin=fmin, fmax=fmax)
    
    return mfcc


In [3]:
#결과 재현을 위한 시드고정
np.random.seed(0)

# CTL 파일 경로
ctl_file = '202301ml_fmcc/fmcc_train.ctl'

# RAW 파일이 저장된 폴더 경로
raw_folder = '202301ml_fmcc/raw16k/train/'

# WAV 파일이 저장될 폴더 경로
wav_folder = '202301ml_fmcc/raw16k/train/wavtrain/'

# CTL 파일 읽기
with open(ctl_file, 'r') as file:
    filelist = [line.strip().replace('\\', '/') + '.raw' for line in file]

#.raw형태의 학습 데이터를 .wav로 바꿔준 후 저장할 폴더 생성
if os.path.exists(wav_folder) == False:
    os.mkdir(wav_folder)
    
# RAW 파일을 WAV 파일로 변환
for fileinfo in filelist:
    raw_filepath = os.path.join(raw_folder, fileinfo)
    wav_folder_name = wav_folder+fileinfo.split('/')[0]
    if os.path.exists(wav_folder_name) == False:
        os.mkdir(wav_folder_name)
    wav_filename = os.path.splitext(fileinfo)[0] + '.wav'
    wav_filepath = os.path.join(wav_folder, wav_filename)
    convert_raw_to_wav(raw_filepath, wav_filepath)

In [4]:
# 남성 음성 데이터셋
males_dataset = []

# 여성 음성 데이터셋
females_dataset = []

#train 데이터의 raw파일을 wav파일로 변환한 경로 저장
folder_path = "202301ml_fmcc/raw16k/train/wavtrain/"
prefix1 = "M"
prefix2 = "F"
extension = "*.wav"

#위의 wav파일을 저장한 경로에서 남성 wav 파일의 데이터를 file_path1에 여자 wav 파일의 데이터를 file_path2에 순차적으로 저장
file_paths1 = glob.glob(folder_path+ "/**/"+prefix1+extension, recursive=True)
file_paths2 = glob.glob(folder_path+ "/**/"+prefix2+extension, recursive=True)

#순차적으로 저장한 남성 wav 파일의 경로명을 males_dataset에 저장
for file_path in file_paths1:
    males_dataset.append(file_path)

#순차적으로 저장한 어성 wav 파일의 경로명을 females_dataset
for file_path in file_paths2:
    females_dataset.append(file_path)


# 위에서 저장한 남성 음성 .wav파일의 경로명을 읽어 .wav 데이터셋에 대해 전처리
males_features = []
for audio_path in males_dataset:
    features = extract_features(audio_path)
    males_features.append(features)
    
# 위에서 저장한 여성 음성 .wav파일의 경로명을 읽어 .wav 데이터셋에 대해 전처리
females_features = []
for audio_path in females_dataset:
    features = extract_features(audio_path)
    females_features.append(features)

# 위에서 만든 남성과 여성 특성 벡터 파일을 합쳐주기 위해 패딩하는 과정
mansize=max([np.shape(i)[1] for i in males_features])
femalesize=max([np.shape(i)[1] for i in females_features])
if(mansize > femalesize):
    max_size= max([np.shape(i)[1] for i in males_features])
else:
    max_size = max([np.shape(i)[1] for i in females_features])
    
#남성, 여성 데이터셋 중 길이가 가장 긴 값으로 패딩을 해줌
man_features = []
for i in males_features:
    current_size = np.shape(i)[1]
    if current_size < max_size:
        padding_size = max_size - current_size
        padding = np.zeros((np.shape(i)[0], padding_size))
        padded_array = np.concatenate((i, padding), axis=1)
        man_features.append(padded_array)
    else:
        man_features.append(i)
        
#패딩을 끝낸 후 합쳐 주기 위해 ndarray로 변환
males_features = np.array(man_features, dtype=object)


woman_features=[]    
for i in females_features:
    current_size = np.shape(i)[1]
    if current_size < max_size:
        padding_size = max_size - current_size
        padding = np.zeros((np.shape(i)[0], padding_size))
        padded_array = np.concatenate((i, padding), axis=1)
        woman_features.append(padded_array)
    else:
        woman_features.append(i)
        
#패딩을 끝낸 후 합쳐 주기 위해 ndarray로 변환
females_features = np.array(woman_features, dtype=object)

#패딩을 끝마친 남성과 여성 특성 데이터셋을 학습을 위해 합침
all_features = np.concatenate((males_features, females_features), axis=0)

#아래의 LDA 전처리를 하기 위해 남성을 0으로 여성을 1로 라벨링한 후 두 label데이터를 합침
males_labels =  np.zeros(len(males_features))
females_labels = np.ones(len(females_features))
all_labels = np.concatenate((males_labels, females_labels), axis=0)

In [5]:
# 데이터 전처리
flattened_features = all_features.reshape(all_features.shape[0], -1)  # 3D 데이터를 2D로 변환

# NaN 값이 있는 경우 0으로 대체
flattened_features = np.nan_to_num(flattened_features, nan=0.0)

# LDA를 통한 특성 변환
n_components=1
lda = LinearDiscriminantAnalysis(n_components=n_components)
lda_features = lda.fit_transform(flattened_features, all_labels)

# # GMM 모델 훈련
n_clusters = 2  # 클러스터 개수 (성별 개수와 일치)
gmm = GaussianMixture(n_components=n_clusters, covariance_type='full', random_state=0)
gmm.fit(lda_features)

#학습한 모델을 file_name으로 저장
file_name = 'gmm_model.pkl'
joblib.dump(gmm, file_name)

#테스트용 코드에서 테스트 데이터의 패딩을 위해 max_szie와 위의 전처리에 사용한 lda 파라미터를 테스트용 코드에서 사용해주기 위해 저장
%store max_size
%store lda

Stored 'max_size' (int)
Stored 'lda' (LinearDiscriminantAnalysis)
